In [ ]:
# | default_exp _cli

In [ ]:
# | export

import asyncio
import multiprocessing
from typing import *

import typer

from fastkafka._components.logger import get_logger, supress_timestamps

from fastkafka import _cli_docs, _cli_testing
from fastkafka._server import run_fastkafka_server

In [ ]:
import os
import time

from typer.testing import CliRunner

from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka._server import terminate_asyncio_process
from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

supress_timestamps(False)
logger = get_logger(__name__, level=20)

In [ ]:
# supress_timestamps()
# logger = get_logger(__name__, level=20)
logger.info("ok")

23-05-12 13:17:25.806 [INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")

In [ ]:
# | export


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(
        multiprocessing.cpu_count(),
        help="Number of FastKafka instances to run, defaults to number of CPU cores.",
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
    kafka_broker: str = typer.Option(
        ...,
        help="kafka_broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastaKafka class.",
    ),
) -> None:
    try:
        asyncio.run(
            run_fastkafka_server(
                num_workers=num_workers, app=app, kafka_broker=kafka_broker
            )
        )
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: run [OPTIONS] APP                                                                                          
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│    --num-workers               INTEGER  Number of FastKafka instances to run, defaults to number of CPU cores.  │
│                                         [default: 64]                                                           │
│ *  --kafka-broker              TEXT     kafka_broker, one of the keys of the kafka_brokers dictionary passed in │
│                                         the constructor of FastaKafka class.                                    │
│                                         [default: None]                                                         │
│                                         [required]                                                              │
│    --install-completion                 Install completion for the current shell.                               │
│    --show-completion                    Show completion for the current shell, to copy it or customize the      │
│                                         installation.                                                           │
│    --help                               Show this message and exit.                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
async with ApacheKafkaBroker() as bootstrap_server:
    os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")

    with generate_app_in_tmp() as app:
        proc = await asyncio.create_subprocess_exec(
            "fastkafka",
            "run",
            "--num-workers",
            f"{2}",
            "--kafka-broker",
            "localhost",
            app,
            stdout=asyncio.subprocess.PIPE,
        )
        time.sleep(5)
        await terminate_asyncio_process(proc)
        outputs, errs = await proc.communicate()

        print(outputs.decode("utf-8"))
        assert proc.returncode == 0, proc.returncode

23-05-12 13:17:39.956 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-05-12 13:17:39.957 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-12 13:17:39.957 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-05-12 13:17:39.958 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-12 13:17:39.959 [INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
23-05-12 13:17:40.682 [INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
23-05-12 13:17:42.536 [INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
23-05-12 13:17:47.940 [INFO] fastkafka._server: terminate_asyncio_process(): Terminating the process 7570...
23-05-12 13:17:50.251 [INFO] fastkafka._server: terminate_asyncio_process(): Process 7570 terminated.
[7574]: 23-05-12 13:17:43.797 [INFO] fastkafka._application.app: set_kafka_broker() :

23-05-12 13:17:52.008 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 6974 terminated.
23-05-12 13:17:52.009 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 6595...
23-05-12 13:17:53.355 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 6595 terminated.


In [ ]:
# | export


_app.add_typer(_cli_docs._docs_app, name="docs")

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["docs", "install_deps", "--help"])

                                                                                                                   
 Usage: root docs install_deps [OPTIONS]                                                                           
                                                                                                                   

Installs dependencies for FastKafka documentation generation

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["docs", "install_deps"])
assert result.exit_code == 0

23-05-12 13:20:16.355 [INFO] fastkafka._components.docs_dependencies: AsyncAPI generator installed


In [ ]:
result = runner.invoke(_app, ["docs", "generate", "--help"])

                                                                                                                   
 Usage: root docs generate [OPTIONS] APP                                                                           
                                                                                                                   

Generates documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created [default: .]                       │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as import_str:
    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

23-05-12 13:20:20.521 [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpkb79wdd3/asyncapi/spec/asyncapi.yml' does not exist.
23-05-12 13:20:20.532 [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpkb79wdd3/asyncapi/spec/asyncapi.yml'
23-05-12 13:20:27.363 [INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
23-05-12 13:20:27.363 [INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmpkb79wdd3/asyncapi/docs.






In [ ]:
result = runner.invoke(_app, ["docs", "serve", "--help"])

                                                                                                                   
 Usage: root docs serve [OPTIONS] APP                                                                              
                                                                                                                   

Generates and serves documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT     root path under which documentation will be created [default: .]                    │
│ --bind             TEXT     Some info [default: 127.0.0.1]                                                      │
│ --port             INTEGER  Some info [default: 8000]                                                           │
│ --help                      Show this message and exit.                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
        "fastkafka",
        "docs",
        "serve",
        "--port=48000",
        app,
        stdout=asyncio.subprocess.PIPE,
    )
    time.sleep(30)
    await terminate_asyncio_process(proc)
    outputs, errs = await proc.communicate()
    assert proc.returncode == 0, proc.returncode
    print(outputs.decode("utf-8"))

23-05-12 13:20:40.411 [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp0yh7v782/asyncapi/spec/asyncapi.yml' does not exist.
23-05-12 13:20:40.423 [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmp0yh7v782/asyncapi/spec/asyncapi.yml'
23-05-12 13:20:47.313 [INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
23-05-12 13:20:47.313 [INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmp0yh7v782/asyncapi/docs.


Serving documentation on http://127.0.0.1:48000
Interupting serving of documentation and cleaning up...



In [ ]:
# | export


_app.add_typer(_cli_testing._testing_app, name="testing")

In [ ]:
result = runner.invoke(_app, ["testing", "install_deps", "--help"])

                                                                                                                   
 Usage: root testing install_deps [OPTIONS]                                                                        
                                                                                                                   

Installs dependencies for FastKafka app testing

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["testing", "install_deps"])
assert result.exit_code == 0